
**Your goal**: wrangle WeRateDogs Twitter data to create interesting and trustworthy analyses and visualizations. The Twitter archive is great, but it only contains very basic tweet information. Additional gathering, then assessing and cleaning is required for "Wow!"-worthy analyses and visualizations.


1. The WeRateDogs Twitter archive. I am giving this file to you, so imagine it as a file on hand. Download this file manually by clicking the following link: twitter_archive_enhanced.csv

2. The tweet image predictions, i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network. This file (image_predictions.tsv) is hosted on Udacity's servers and should be downloaded programmatically using the Requests library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv]

3. Each tweet's retweet count and favorite ("like") count at minimum, and any additional data you find interesting. Using the tweet IDs in the WeRateDogs Twitter archive, query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called tweet_json.txt file. Each tweet's JSON data should be written to its own line. Then read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count. Note: do not include your Twitter API keys, secrets, and tokens in your project submission.

Detect and document at least
* **Eight quality issues**
* **Two tidiness issues**


* **three insights** 
* **one visualization**

* Create a 300-600 word written report called wrangle_report.pdf or wrangle_report.html that briefly describes your wrangling efforts. This is to be framed as an internal document.

* Create a 250-word-minimum written report called act_report.pdf or act_report.html that communicates the insights and displays the visualization(s) produced from your wrangled data. This is to be framed as an external document, like a blog post or magazine article, for example.


## Basic wrangling strategy

1. Gather all data sets, document issues pertinent to every dataset in the process.
2. Merge all data sets.
3. Remove duplicate and redundant data.
4. Resolve tidiness issues.
5. Resolve quality issues.


In [1]:
import requests as rq
import pandas as pd
import numpy as np
import io
import json

# Step 1. Gather data

## Dataset 1 - Enhanced twitter archive
### Read the enhanced twitter archive into a dataframe

In [2]:
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
twitter_archive.head(15)

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0   892420643555336193                    NaN                  NaN   
1   892177421306343426                    NaN                  NaN   
2   891815181378084864                    NaN                  NaN   
3   891689557279858688                    NaN                  NaN   
4   891327558926688256                    NaN                  NaN   
5   891087950875897856                    NaN                  NaN   
6   890971913173991426                    NaN                  NaN   
7   890729181411237888                    NaN                  NaN   
8   890609185150312448                    NaN                  NaN   
9   890240255349198849                    NaN                  NaN   
10  890006608113172480                    NaN                  NaN   
11  889880896479866881                    NaN                  NaN   
12  889665388333682689                    NaN                  NaN   
13  889638837579907072                    NaN                  NaN   
14  889531135344209921                    NaN                  NaN   

                    timestamp  \
0   2017-08-01 16:23:56 +0000   
1   2017-08-01 00:17:27 +0000   
2   2017-07-31 00:18:03 +0000   
3   2017-07-30 15:58:51 +0000   
4   2017-07-29 16:00:24 +0000   
5   2017-07-29 00:08:17 +0000   
6   2017-07-28 16:27:12 +0000   
7   2017-07-28 00:22:40 +0000   
8   2017-07-27 16:25:51 +0000   
9   2017-07-26 15:59:51 +0000   
10  2017-07-26 00:31:25 +0000   
11  2017-07-25 16:11:53 +0000   
12  2017-07-25 01:55:32 +0000   
13  2017-07-25 00:10:02 +0000   
14  2017-07-24 17:02:04 +0000   

                                               source  \
0   <a href="http://twitter.com/download/iphone" r...   
1   <a href="http://twitter.com/download/iphone" r...   
2   <a href="http://twitter.com/download/iphone" r...   
3   <a href="http://twitter.com/download/iphone" r...   
4   <a href="http://twitter.com/download/iphone" r...   
5   <a href="http://twitter.com/download/iphone" r...   
6   <a href="http://twitter.com/download/iphone" r...   
7   <a href="http://twitter.com/download/iphone" r...   
8   <a href="http://twitter.com/download/iphone" r...   
9   <a href="http://twitter.com/download/iphone" r...   
10  <a href="http://twitter.com/download/iphone" r...   
11  <a href="http://twitter.com/download/iphone" r...   
12  <a href="http://twitter.com/download/iphone" r...   
13  <a href="http://twitter.com/download/iphone" r...   
14  <a href="http://twitter.com/download/iphone" r...   

                                                 text  retweeted_status_id  \
0   This is Phineas. He's a mystical boy. Only eve...                  NaN   
1   This is Tilly. She's just checking pup on you....                  NaN   
2   This is Archie. He is a rare Norwegian Pouncin...                  NaN   
3   This is Darla. She commenced a snooze mid meal...                  NaN   
4   This is Franklin. He would like you to stop ca...                  NaN   
5   Here we have a majestic great white breaching ...                  NaN   
6   Meet Jax. He enjoys ice cream so much he gets ...                  NaN   
7   When you watch your owner call another dog a g...                  NaN   
8   This is Zoey. She doesn't want to be one of th...                  NaN   
9   This is Cassie. She is a college pup. Studying...                  NaN   
10  This is Koda. He is a South Australian decksha...                  NaN   
11  This is Bruno. He is a service shark. Only get...                  NaN   
12  Here's a puppo that seems to be on the fence a...                  NaN   
13  This is Ted. He does his best. Sometimes that'...                  NaN   
14  This is Stuart. He's sporting his favorite fan...                  NaN   

    retweeted_status_user_id retweeted_status_timestamp  \
0                        NaN                        NaN   
1                        NaN                        NaN   
2                        

In [4]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

## Dataset 2 - Image predictions

### Download the tweet image predictions

In [5]:
r = rq.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')
r.status_code

200

In [6]:
r.headers['content-type']

'text/tab-separated-values; charset=utf-8'

In [7]:
r.encoding

'utf-8'

#### Load the received .tsv file into a dataframe

In [ ]:
# load image predictions dataset from a local file
image_predictions = pd.DataFrame.from_csv('image-predictions.tsv', sep='\t')

In [8]:
# load image predictions dataset from the URL
image_predictions = pd.DataFrame.from_csv(io.StringIO(r.content.decode(r.encoding)), sep='\t')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  


In [9]:
image_predictions.head()

jpg_url  img_num  \
tweet_id                                                                       
666020888022790149  https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg        1   
666029285002620928  https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg        1   
666033412701032449  https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg        1   
666044226329800704  https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg        1   
666049248165822465  https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg        1   

                                        p1   p1_conf  p1_dog  \
tweet_id                                                       
666020888022790149  Welsh_springer_spaniel  0.465074    True   
666029285002620928                 redbone  0.506826    True   
666033412701032449         German_shepherd  0.596461    True   
666044226329800704     Rhodesian_ridgeback  0.408143    True   
666049248165822465      miniature_pinscher  0.560311    True   

                                    p2   p2_conf  p2_dog                   p3  \
tweet_id                                                                        
666020888022790149              collie  0.156665    True    Shetland_sheepdog   
666029285002620928  miniature_pinscher  0.074192    True  Rhodesian_ridgeback   
666033412701032449            malinois  0.138584    True           bloodhound   
666044226329800704             redbone  0.360687    True   miniature_pinscher   
666049248165822465          Rottweiler  0.243682    True             Doberman   

                     p3_conf  p3_dog  
tweet_id                              
666020888022790149  0.061428    True  
666029285002620928  0.072010    True  
666033412701032449  0.116197    True  
666044226329800704  0.222752    True  
666049248165822465  0.154629    True

In [10]:
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2075 entries, 666020888022790149 to 892420643555336193
Data columns (total 11 columns):
jpg_url    2075 non-null object
img_num    2075 non-null int64
p1         2075 non-null object
p1_conf    2075 non-null float64
p1_dog     2075 non-null bool
p2         2075 non-null object
p2_conf    2075 non-null float64
p2_dog     2075 non-null bool
p3         2075 non-null object
p3_conf    2075 non-null float64
p3_dog     2075 non-null bool
dtypes: bool(3), float64(3), int64(1), object(4)
memory usage: 152.0+ KB


### Merge twitter archive and image predictions

In [11]:
merged_df = pd.merge(twitter_archive,image_predictions,how='inner',on='tweet_id')

In [12]:
merged_df.head()

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0  892420643555336193                    NaN                  NaN   
1  892177421306343426                    NaN                  NaN   
2  891815181378084864                    NaN                  NaN   
3  891689557279858688                    NaN                  NaN   
4  891327558926688256                    NaN                  NaN   

                   timestamp  \
0  2017-08-01 16:23:56 +0000   
1  2017-08-01 00:17:27 +0000   
2  2017-07-31 00:18:03 +0000   
3  2017-07-30 15:58:51 +0000   
4  2017-07-29 16:00:24 +0000   

                                              source  \
0  <a href="http://twitter.com/download/iphone" r...   
1  <a href="http://twitter.com/download/iphone" r...   
2  <a href="http://twitter.com/download/iphone" r...   
3  <a href="http://twitter.com/download/iphone" r...   
4  <a href="http://twitter.com/download/iphone" r...   

                                                text  retweeted_status_id  \
0  This is Phineas. He's a mystical boy. Only eve...                  NaN   
1  This is Tilly. She's just checking pup on you....                  NaN   
2  This is Archie. He is a rare Norwegian Pouncin...                  NaN   
3  This is Darla. She commenced a snooze mid meal...                  NaN   
4  This is Franklin. He would like you to stop ca...                  NaN   

   retweeted_status_user_id retweeted_status_timestamp  \
0                       NaN                        NaN   
1                       NaN                        NaN   
2                       NaN                        NaN   
3                       NaN                        NaN   
4                       NaN                        NaN   

                                       expanded_urls  ...  img_num  \
0  https://twitter.com/dog_rates/status/892420643...  ...        1   
1  https://twitter.com/dog_rates/status/892177421...  ...        1   
2  https://twitter.com/dog_rates/status/891815181...  ...        1   
3  https://twitter.com/dog_rates/status/891689557...  ...        1   
4  https://twitter.com/dog_rates/status/891327558...  ...        2   

            p1   p1_conf p1_dog                  p2   p2_conf p2_dog  \
0       orange  0.097049  False               bagel  0.085851  False   
1    Chihuahua  0.323581   True            Pekinese  0.090647   True   
2    Chihuahua  0.716012   True            malamute  0.078253   True   
3  paper_towel  0.170278  False  Labrador_retriever  0.168086   True   
4       basset  0.555712   True    English_springer  0.225770   True   

                            p3   p3_conf p3_dog  
0                       banana  0.076110  False  
1                     papillon  0.068957   True  
2                       kelpie  0.031379   True  
3                      spatula  0.040836  False  
4  German_short-haired_pointer  0.175219   True  

[5 rows x 28 columns]

## Dataset 3: Twitter via API

### Set up Twitter API via tweepy

In [ ]:
import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
api.wait_on_rate_limit = True

### Download WeRateDogs Twitter archive. Takes around 30 mins.

In [ ]:
# create an empty array to store dictionaries retrieved via API
tweets = []
missing_tweets = []

# use tweet_id's from our dataframe to retrieve original tweets
for i in merged_df.tweet_id:
    try:
        tweets.append (api.get_status(i, tweet_mode='extended'))
    except:
        missing_tweets.append(i)
        print ('tweet #', i, ' could not be located')
            
# write downloaded tweets to a json file and store it locally            
with open('tweets.json', 'w') as outfile:
    json.dump(tweets, outfile)

In [ ]:
missing_tweets

### Read the downloaded and saved archive from a local .json file

In [13]:
tweets = pd.read_json ('tweets.json')

### Merge tweets with two previous datasets

In [14]:
# rename id columns to identical names
tweets.rename(columns={'id':'tweet_id'},inplace=True)

# merge two dataframes on the 'tweet_id' column
we_rate_dogs = pd.merge(merged_df,tweets, how='inner', on='tweet_id')

# Step 2. Assess data

In [15]:
we_rate_dogs.head()

tweet_id  in_reply_to_status_id_x  in_reply_to_user_id_x  \
0  892420643555336193                      NaN                    NaN   
1  892177421306343426                      NaN                    NaN   
2  891815181378084864                      NaN                    NaN   
3  891689557279858688                      NaN                    NaN   
4  891327558926688256                      NaN                    NaN   

                   timestamp  \
0  2017-08-01 16:23:56 +0000   
1  2017-08-01 00:17:27 +0000   
2  2017-07-31 00:18:03 +0000   
3  2017-07-30 15:58:51 +0000   
4  2017-07-29 16:00:24 +0000   

                                            source_x  \
0  <a href="http://twitter.com/download/iphone" r...   
1  <a href="http://twitter.com/download/iphone" r...   
2  <a href="http://twitter.com/download/iphone" r...   
3  <a href="http://twitter.com/download/iphone" r...   
4  <a href="http://twitter.com/download/iphone" r...   

                                                text  retweeted_status_id  \
0  This is Phineas. He's a mystical boy. Only eve...                  NaN   
1  This is Tilly. She's just checking pup on you....                  NaN   
2  This is Archie. He is a rare Norwegian Pouncin...                  NaN   
3  This is Darla. She commenced a snooze mid meal...                  NaN   
4  This is Franklin. He would like you to stop ca...                  NaN   

   retweeted_status_user_id retweeted_status_timestamp  \
0                       NaN                        NaN   
1                       NaN                        NaN   
2                       NaN                        NaN   
3                       NaN                        NaN   
4                       NaN                        NaN   

                                       expanded_urls  ...  lang  place  \
0  https://twitter.com/dog_rates/status/892420643...  ...    en   None   
1  https://twitter.com/dog_rates/status/892177421...  ...    en   None   
2  https://twitter.com/dog_rates/status/891815181...  ...    en   None   
3  https://twitter.com/dog_rates/status/891689557...  ...    en   None   
4  https://twitter.com/dog_rates/status/891327558...  ...    en   None   

  possibly_sensitive possibly_sensitive_appealable retweet_count retweeted  \
0              False                         False          7979     False   
1              False                         False          5925     False   
2              False                         False          3913     False   
3              False                         False          8132     False   
4              False                         False          8812     False   

  retweeted_status                                           source_y  \
0              NaN  <a href="http://twitter.com/download/iphone" r...   
1              NaN  <a href="http://twitter.com/download/iphone" r...   
2              NaN  <a href="http://twitter.com/download/iphone" r...   
3              NaN  <a href="http://twitter.com/download/iphone" r...   
4              NaN  <a href="http://twitter.com/download/iphone" r...   

   truncated                                               user  
0      False  {'id': 4196983835, 'id_str': '4196983835', 'na...  
1      False  {'id': 4196983835, 'id_str': '4196983835', 'na...  
2      False  {'id': 4196983835, 'id_str': '4196983835', 'na...  
3      False  {'id': 4196983835, 'id_str': '4196983835', 'na...  
4      False  {'id': 4196983835, 'id_str': '4196983835', 'na...  

[5 rows x 55 columns]

In [16]:
we_rate_dogs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2061 entries, 0 to 2060
Data columns (total 55 columns):
tweet_id                         2061 non-null int64
in_reply_to_status_id_x          23 non-null float64
in_reply_to_user_id_x            23 non-null float64
timestamp                        2061 non-null object
source_x                         2061 non-null object
text                             2061 non-null object
retweeted_status_id              74 non-null float64
retweeted_status_user_id         74 non-null float64
retweeted_status_timestamp       74 non-null object
expanded_urls                    2061 non-null object
rating_numerator                 2061 non-null int64
rating_denominator               2061 non-null int64
name                             2061 non-null object
doggo                            2061 non-null object
floofer                          2061 non-null object
pupper                           2061 non-null object
puppo                            2061 

In [17]:
print (len(we_rate_dogs.columns))

55


## 1. Duplicate, zero and redundant data
First let's assess whether there are any columns that duplicate data, contain no data whatsoever or that are irrelevant for the purposes of our analysis. Let's make empty lists which we will populate over the course of our assessment and drop remove them from the dataframe.

In [18]:
duplicate_columns = []
zero_columns = []
redundant_columns = []

Summing data in columns highlights some of the empty variables.

In [19]:
we_rate_dogs.sum(axis=0)

tweet_id                                                       8098880034596373798
in_reply_to_status_id_x                                                1.60497e+19
in_reply_to_user_id_x                                                  9.65306e+10
timestamp                        2017-08-01 16:23:56 +00002017-08-01 00:17:27 +...
source_x                         <a href="http://twitter.com/download/iphone" r...
text                             This is Phineas. He's a mystical boy. Only eve...
retweeted_status_id                                                    5.63835e+19
retweeted_status_user_id                                               7.87462e+17
expanded_urls                    https://twitter.com/dog_rates/status/892420643...
rating_numerator                                                             25273
rating_denominator                                                           21671
name                             PhineasTillyArchieDarlaFranklinNoneJaxNoneZoey...
dogg

In [20]:
# make list of all zero-value columns
zero_columns.extend(['contributors','coordinates','favorited','geo','is_quote_status','possibly_sensitive','possibly_sensitive_appealable','retweeted','truncated'])

It seems that there is only one record in the 'place' column. What is it?

In [21]:
we_rate_dogs[we_rate_dogs.place.notnull()].place

686    {'id': '7356b662670b2c31', 'url': 'https://api...
Name: place, dtype: object

Let's store this place in a separate variable and remove the column.

In [23]:
wrd_place = we_rate_dogs.iloc[686]
zero_columns.append('place')

In this analysis we will not engage in understanding social context of this account, therefore all information pertaining to retweets and replies can be considered redundant.

In [24]:
redundant_columns.extend(['in_reply_to_status_id_x', 'in_reply_to_user_id_x',
'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp',
'in_reply_to_screen_name','in_reply_to_status_id_y', 'in_reply_to_status_id_str',
'in_reply_to_user_id_y', 'in_reply_to_user_id_str','retweeted_status'])

In [25]:
# are source_x and source_y columns identical?
# sum the number of times records differ and subtract from this the length of the whole dataset
# '0' would indicate that all the records in these columns are identical
print (sum(we_rate_dogs.source_x == we_rate_dogs.source_y) - len(we_rate_dogs))

0


In [26]:
duplicate_columns.append('source_y')

In [27]:
duplicate_columns

['source_y']

In [ ]:
we_rate_dogs.drop(columns=duplicate_columns, inplace=True)
we_rate_dogs.drop(columns=zero_columns, inplace=True)
we_rate_dogs.drop(columns=redundant_columns,inplace=True)

In [ ]:
print ('number if columns in the dataset: ', len(we_rate_dogs.columns))

In [ ]:
we_rate_dogs.info()

### 2. Incorrect data types.

Some of the columns have 

### Define 
`display_text_range` column can be effectively reduced to a single `int` rather than a list.

In [ ]:
sum(we_rate_dogs.display_text_range.apply(lambda x: 1 if x[0] > 0 else 0))

#### Code

extract the relevant value, put it into a new column and drop the old column

In [ ]:
we_rate_dogs['display_text_end'] = we_rate_dogs.display_text_range.apply(lambda x: x[1]).astype(int)
we_rate_dogs.drop(columns='display_text_range', inplace=True)

#### Test
check if our new colum is of `int64` type

In [ ]:
print (type(we_rate_dogs.display_text_end[0]))
we_rate_dogs.head()

The `user` field contains a number of interesting fields. Let's unpack some of those into our dataframe.

### Define
Unpack `followers_count` field from `user` into separate column

#### Code
Using `apply` function - unpack the values.

In [ ]:
we_rate_dogs['followers'] = we_rate_dogs.user.apply(lambda x: x['followers_count'])

#### Test
Check the new column is in place and that it's `int64` type.

In [ ]:
we_rate_dogs.info()

### Define
Unpack `favourites_count` from within `user` field into a separate column.

In [ ]:
we_rate_dogs['total_favourites'] = we_rate_dogs.user.apply(lambda x: x['favourites_count'])

In [ ]:
print (we_rate_dogs['total_favourites'].min())
print (we_rate_dogs['total_favourites'].mean())
print (we_rate_dogs['total_favourites'].max())

### Define

There should be a limited set of appications used to post tweets.

In [ ]:
# how many various sources were used to post tweets?
we_rate_dogs.source_x.unique()

#### Code
This kind of data is best represented as a categorical variable.

In [ ]:
we_rate_dogs.source_x = we_rate_dogs.source_x.apply(lambda x: x.split('>')[1].split('<')[0]).astype('category')

####  Test
Check unique values in the column.

In [ ]:
we_rate_dogs.source_x.unique()

### Define
Hashtags are nested deep inside dictionaries. To be useful for analysis - they need to be extracted into a separate columnb

#### Code

extract hashtags using the `apply` method, put them in a separate column and cast them into `category` type variable.

In [ ]:
we_rate_dogs['hashtags'] = we_rate_dogs.entities.apply(lambda x: x['hashtags'])

In [ ]:
def unpack_hashtags(x):
    try:
        return x[0]['text']
    except:
        return None
        
we_rate_dogs.hashtags = we_rate_dogs.hashtags.apply(lambda x: unpack_hashtags(x))

In [ ]:
we_rate_dogs.hashtags.unique()

With this few hashtags they are better as a 'category' type variable.

In [ ]:
we_rate_dogs.hashtags = we_rate_dogs.hashtags.astype('category')

#### Test
Check that the `hashtags` colummn is a `category` type.

In [ ]:
we_rate_dogs.info()

### Define

Same as hashtags - `url`s are nested deep inside dictionaries. They need to be extracted into a separate column. 

#### Code
Using `apply` function with a short lambda expression - extract the `url` into a new column.

In [ ]:
we_rate_dogs['url'] = we_rate_dogs.entities.apply(lambda x: x['media'][0]['url'])

#### Test
Check the new `url` column has the right information.

In [ ]:
we_rate_dogs.head()

In [ ]:
we_rate_dogs.info()

### Define

`timestamp` and `crated_at` columns are probably identical. If that's the case - one needs to be removed.

#### Code

In [ ]:
# first - convert timestamp into datetime format
we_rate_dogs.timestamp = pd.to_datetime(we_rate_dogs.timestamp)

#### Test

In [ ]:
# strip both fileds of time zone
we_rate_dogs['timestamp'] = we_rate_dogs['timestamp'].apply(lambda x: x.replace(tzinfo=None))
we_rate_dogs['created_at'] = we_rate_dogs['created_at'].apply(lambda x: x.replace(tzinfo=None))

In [ ]:
# subtract one from the other and sum the differences to see if there's anything different between the two
(we_rate_dogs['timestamp'] - we_rate_dogs['created_at']).unique()

In [ ]:
# drop the created_at column
we_rate_dogs.drop(columns='created_at',inplace=True)

In [ ]:
we_rate_dogs.info()

### Define

`doggo`, `floofer`, `pupper`, `puppo` columns are better represented as `bool`s.

#### Code

In [ ]:
def to_bool(x):
    if x=='None':
        return False
    else:
        return True
    
we_rate_dogs.doggo = we_rate_dogs.doggo.apply(to_bool)
we_rate_dogs.floofer = we_rate_dogs.floofer.apply(to_bool)
we_rate_dogs.pupper = we_rate_dogs.pupper.apply(to_bool)
we_rate_dogs.puppo = we_rate_dogs.puppo.apply(to_bool)

#### Test
Check if the columns have correct data types in them.

In [ ]:
we_rate_dogs.info()

### Define
Convert `lang` column into category.

#### Code

In [ ]:
we_rate_dogs.lang = we_rate_dogs.lang.astype('category')

### Test

In [ ]:
we_rate_dogs.info()

## Analysis part

In [ ]:
print ('start date: ', we_rate_dogs.timestamp.min())
print ('end date: ', we_rate_dogs.timestamp.max())
duration = we_rate_dogs.timestamp.max()- we_rate_dogs.timestamp.min()
print ('duration: ', duration)
print ('average tweets per day: ', len(we_rate_dogs)/duration.days)

In [ ]:
type(duration.days)